# Třetí cvičení - jazykové modely, perplexita

## 1) Příprava dat

- Soubory TEXTCZ1 a TEXTEN1 přeuspořádejte tak, aby každá věta byla na jednom řádku (jako oddělovač vět použijte tečku, otazník a vykřičník).
- Všechny slova převeďte na lowercase

In [11]:
import re, string
from collections import Counter
from math import log
import math

## Unigramový model
- Vypočítejte pravděpodobnost každého ze slov
- Vypočítejte entropii H(X) a perplexitu G(X) modelu

$H(X) = - \sum_{x}p(x)log{_2}p(x)$

$G(X) = 2^{H(X)}$

In [14]:
# Nejprve vytvořte unigramový model pro větu: "<s> Dnes je hezký den"
sent = "<s> Dnes je hezký den"
...
entropy = ...
perplexity = ...

# Očekávaný výsledek: 
# entropy = 2.321928094887362
# perplexity = 4.999999999999999



In [15]:
# Nyní pro CZ a EN korpusy

In [12]:
# Načtení souborů CZ, EN
....
        

In [13]:
# Rozdělení textů na věty, odstranění interpunkce a přidání znaku začátku věty
....

In [16]:
# Anglický unigramový model

...

en_entropy = ...
en_perplexity = ...

print(f"Entropy: {en_entropy} Perplexity: {en_perplexity}")
# Očekávaný výsledek: 
# en_entropy = 9.153156076444484
# en_perplexity = 569.3437191681389

Entropy: 9.153156076444484 Perplexity: 569.3437191681389


In [17]:
# Český unigramový model

...

cz_entropy = ...
cz_perplexity = ...

print(f"Entropy: {cz_entropy} Perplexity: {cz_perplexity}")
# Očekávaný výsledek: 
# en_entropy = 11.866640196670001
# en_perplexity = 3734.346796191333


Entropy: 11.866640196670001 Perplexity: 3734.346796191333


## Bigramový model
- obdobně vytvořte bigramový model a všechny dvojice slov hledejte vždy pouze ve větě (nikoliv mezi větami)
- vypočítejte entropii a perplexitu

$H(B|A) = - \sum_{a \in A, b \in B}P(a,b)log{_2}P(b|a)$ 

$G(X) = 2^{H(B|A)}$

In [21]:
# Nejprve vytvořte bigramový model pro větu: "<s> Dnes je hezký den </s>"
sent = ["<s> dnes respektive dnes možná </s>"]

...

entropy = ...
perplexity = ...

print(f"Entropy: {entropy} Perplexity: {perplexity}")
# Očekávaný výsledek: 
# entropy = 0.4
# perplexity = 1.3195079107728942

Entropy: 0.4 Perplexity: 1.3195079107728942


In [22]:
# Nyní pro jednotlivé korpusy, nejprve pro CZ a pak pro EN

# Přidání koncového znaku </s> do vět
...

In [23]:
# Rozdělení na slova, tvorba slovníku a výpočet frekvencí
...

In [24]:

....
en_entropy = ...
en_perplexity = ...

print(f"Entropy: {en_entropy} Perplexity: {en_perplexity}")

# Očekávaný výsledek: 
# en_entropy = 5.380173547272233
# en_perplexity = 41.64794906297528

Entropy: 5.380173547272145 Perplexity: 41.647949062972735


In [25]:
...
cz_entropy = ...
cz_perplexity = ...

print(f"Entropy: {cz_entropy} Perplexity: {cz_perplexity}")

# Očekávaný výsledek: 
# cz_entropy = 4.624183207248371
# cz_perplexity = 24.6614070105357

Entropy: 4.624183207248371 Perplexity: 24.6614070105357


In [ ]:
#SRILM příkazy
#ngram-count -text TEXTEN_unigram.txt -order 2 -write czbigram.count -unk
#ngram-count -gt1max 0 -gt2max 0 -read czbigram.count -order 2 -lm czbigram.lm
#ngram -ppl TEXTEN_unigram.txt -order 2 -lm czbigram.lm >> czbigram.ppl

